In [1]:
import ollama
import chromadb
import pandas as pd

/tmp/ipykernel_3169404/1963182100.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('data/recycling guide.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
    
data = {'text': lines}
df = pd.DataFrame(data)

df.head()

,text
0,재활용품은 재질별로 분류하여 분리수거함 및 별도의 전용수거함으로 배출한다.
1,대형 폐가전제품은 무상 방문 수거를 신청한다. 소형 폐가전제품도 같이 수거 가능하다.
2,"대형폐기물은 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재..."
3,음식물류폐기물는 RFID 기반 음식물 폐기장을 사용하거나 종량제봉투에 담아 배출한다...
4,"유해폐기물로 구분된 폐전건지, 형광등, 폐의약품은 전용수거함을 이용한다."


In [3]:
df.tail()

,text
243,식품보조제 콜라겐가루가 남았는데 어떻게 버려야 하나요? 일반쓰레기로 버려야 하나요 ...
244,섬유유연제가 걸쭉하니 굳어가네요. 남은 양이 꽤 많은데 하수구에 버리자니 막힐 것 ...
245,다 쓴 프린터잉크 (페카트리지)는 어떻게 배출하나요?? 저희 읍에서는 종량제봉투에 ...
246,최근 아이스팩이 심각한 수질오염을 일으킬 수 있다는 얘기를 접했습니다. 단순히 종량...
247,"택배 올 때 비닐로 된 포장재가 있습니다. 뽁뽁이 말고요~ 연두빛 비닐인데, 산화 ..."


In [4]:
num_rows = df.shape[0]
print("DataFrame의 줄 개수:", num_rows)

DataFrame의 줄 개수: 248


In [5]:
#client = chromadb.Client()
client = chromadb.PersistentClient(path="DB_recyc")
collection = client.get_or_create_collection(name="recycling_ollama")

docs = df["text"].tolist()

for i, d in enumerate(docs):
  response = ollama.embeddings(model="all-minilm", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

In [6]:
# an example prompt
prompt = "자전거는 어떻게 버려?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="all-minilm"
)

results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=5
)

res = "\n".join(str(item) for item in results['documents'][0])

print(res)

연탄재는 마대에 담아 배출한다.
휴지는 재활용이 가능한가요? 이물질이 묻은 휴지는 종량제봉투로 배출합니다.
나무류인 주걱은 종량제 봉투로 배출한다.
나무류인 장식품은 종량제 봉투로 배출한다.
나무류인 액자는 종량제 봉투로 배출한다.


In [7]:
sys_prompt = f'''
너는 재활용 전문가로 활동하고 있어.
사람들이 재활용 방법을 물어보면 너는 주어진 Context를 바탕으로 자세하게 답해줘야해.
너는 한글로 대답을 해야돼.

Context: {res}
'''

prompt = '자전거는 어떻게 버려?'

messages = sys_prompt + prompt

output = ollama.generate(
  model="EEVE",
  prompt=messages
)

print(output['response'])

 재활용이 가능해요?

자전거를 재활용하려면, 다음 단계를 따라야 해요:
1. 자전거의 상태를 평가하세요. 제대로 작동하는지 확인하고 손상되지 않았는지 확인해야 합니다. 상태가 양호한 경우 재사용하거나 판매할 수 있어요.
2. 부품이 필요하신가요? 수리나 업그레이드를 위해 일부 부품을 제거하고 보관하세요.
3. 사용 가능한 자전거를 기부하세요. 지역 자선단체, 커뮤니티 센터 또는 다른 자전거 애호가에게 기증하면 새 주인을 찾을 수 있어요.
4. 부품이 필요 없으시다면, 폐기물로 처리하세요. 구멍이 뚫린 플라스틱과 같은 일부 부품은 재활용될 수 있지만, 알루미늄이나 강철 프레임은 보통 지역 자원 회수 센터에서 금속으로 수집돼요.
5. 자전거의 나머지 부품을 처리하세요. 고무 타이어와 케이블 같은 다른 재료들은 종량제 봉투로 배출해야 할 수도 있어요.
6. 자전거를 판매할 생각이라면, 온라인으로 구하거나 지역 광고 플랫폼에서 광고를 올려보세요.
7. 마지막으로, 재활용 센터에 연락해 남은 부품이나 자전거의 처리 방법에 대해 문의하세요. 그들은 특정 지역에 적용되는 규칙을 안내해 줄 거예요.
